<a href="https://colab.research.google.com/github/medsellufbc/BMI/blob/main/SEIRDV%20Dashboard_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 32.7 MB/s eta 0:00:00


In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Sample data
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

# Initialize the app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Fruit Sales Dashboard"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[
            {'label': 'San Francisco', 'value': 'SF'},
            {'label': 'Montreal', 'value': 'Montreal'}
        ],
        value='SF'
    ),
    dcc.Graph(id='bar-chart')
])

# Callback to update the bar chart based on the selected city
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('city-dropdown', 'value')]
)
def update_chart(selected_city):
    filtered_df = df[df['City'] == selected_city]
    fig = px.bar(filtered_df, x='Fruit', y='Amount', color='Fruit', barmode='group')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<ipython-input-2-867ad7d660e3>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-867ad7d660e3>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


<IPython.core.display.Javascript object>

In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Sample data
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [40, 18, 35, 25, 40, 35],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

# Initialize the app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Fruit Sales Dashboard"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[
            {'label': 'San Francisco', 'value': 'SF'},
            {'label': 'Montreal', 'value': 'Montreal'}
        ],
        value='SF'
    ),
    dcc.Graph(id='bar-chart')
])

# Callback to update the bar chart based on the selected city
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('city-dropdown', 'value')]
)
def update_chart(selected_city):
    filtered_df = df[df['City'] == selected_city]
    fig = px.bar(filtered_df, x='Fruit', y='Amount', color='Fruit', barmode='group')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [8]:
!pip install dash plotly

In [9]:
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

# SEIRDV model parameters
alpha = 0.2
gamma1 = 0.1
gamma2 = 0.1
gamma3 = 0.1
rho1 = 0.05
rho2 = 0.05
chi = 0.03
kappa = 0.01
lambda1 = 0.3
lambda2 = 0.2
lambda3 = 0.1
delta = 0.01  # Vaccination rate
tau = 0.1  # Vaccine efficacy
sigma1 = sigma2 = sigma3 = sigma4 = sigma5 = sigma6 = sigma7 = sigma8 = 0.01

# Simulation settings
T = 356  # total time in days
dt = 0.1  # time step
n_steps = int(T / dt)

# Stochastic Runge-Kutta (SRK) method
def SRK2_step(f, x, dt, sigma, dB):
    k1 = dt * f(x)
    k2 = dt * f(x + k1) + sigma * dB
    return x + (k1 + k2) / 2

# Differential equations
def SEIRDV_model(x):
    S, E, I1, I2, I3, R, D, V = x

    dS = - (lambda1 * I1 + lambda2 * I2 + lambda3 * I3) * S
    dE = (lambda1 * I1 + lambda2 * I2 + lambda3 * I3) * S - alpha * E - kappa * E + tau * V * (lambda1 * I1 + lambda2 * I2 + lambda3 * I3)
    dI1 = alpha * E - gamma1 * I1 - rho1 * I1 - kappa * I1
    dI2 = rho1 * I1 - gamma2 * I2 - rho2 * I2 - kappa * I2
    dI3 = rho2 * I2 - gamma3 * I3 - chi * I3 - kappa * I3
    dR = gamma1 * I1 + gamma2 * I2 + gamma3 * I3 - kappa * R
    dD = chi * I3
    dV = S * delta - tau * V * (lambda1 * I1 + lambda2 * I2 + lambda3 * I3) - kappa * V

    return np.array([dS, dE, dI1, dI2, dI3, dR, dD, dV])

# Create the Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("SEIRDV Model Simulation Dashboard"),
    html.Div([
        html.Div([
            html.Label("Initial Susceptible (S0)"),
            dcc.Input(id='S0-input', type='number', value=0.9, step=0.01),
            html.Label("Initial Exposed (E0)"),
            dcc.Input(id='E0-input', type='number', value=0.05, step=0.01),
            html.Label("Initial Infected 1 (I1_0)"),
            dcc.Input(id='I1_0-input', type='number', value=0.01, step=0.01),
            html.Label("Initial Infected 2 (I2_0)"),
            dcc.Input(id='I2_0-input', type='number', value=0.01, step=0.01),
            html.Label("Initial Infected 3 (I3_0)"),
            dcc.Input(id='I3_0-input', type='number', value=0.01, step=0.01),
            html.Label("Initial Recovered (R0)"),
            dcc.Input(id='R0-input', type='number', value=0.0, step=0.01),
            html.Label("Initial Deceased (D0)"),
            dcc.Input(id='D0-input', type='number', value=0.0, step=0.01),
            html.Label("Initial Vaccinated (V0)"),
            dcc.Input(id='V0-input', type='number', value=0.02, step=0.01),
            html.Button('Run Simulation', id='run-button', n_clicks=0)
        ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            dcc.Graph(id='seirdv-graph')
        ], style={'width': '75%', 'display': 'inline-block'})
    ])
])

@app.callback(
    Output('seirdv-graph', 'figure'),
    [Input('run-button', 'n_clicks')],
    [State('S0-input', 'value'),
     State('E0-input', 'value'),
     State('I1_0-input', 'value'),
     State('I2_0-input', 'value'),
     State('I3_0-input', 'value'),
     State('R0-input', 'value'),
     State('D0-input', 'value'),
     State('V0-input', 'value')]
)
def update_graph(n_clicks, S0, E0, I1_0, I2_0, I3_0, R0, D0, V0):
    initial_conditions = np.array([S0, E0, I1_0, I2_0, I3_0, R0, D0, V0])

    # Initialize arrays to store results
    results = np.zeros((n_steps, 8))
    results[0] = initial_conditions

    # Brownian motion increments
    dB = np.random.normal(0, np.sqrt(dt), (n_steps, 8))

    # Simulation loop
    for t in range(1, n_steps):
        x = results[t-1]
        results[t] = SRK2_step(SEIRDV_model, x, dt, np.array([sigma1, sigma2, sigma3, sigma4, sigma5, sigma6, sigma7, sigma8]), dB[t])

    time = np.linspace(0, T, n_steps)
    traces = []
    labels = ['S', 'E', 'I1', 'I2', 'I3', 'R', 'D', 'V']

    for i, label in enumerate(labels):
        traces.append(go.Scatter(
            x=time,
            y=results[:, i],
            mode='lines',
            name=label
        ))

    return {
        'data': traces,
        'layout': go.Layout(
            title='SEIRDV Model Simulation (Runge-Kutta Method)',
            xaxis={'title': 'Days'},
            yaxis={'title': 'Proportion of Population'},
            hovermode='closest'
        )
    }

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>